In [10]:
import json
import numpy as np

In [11]:
def write_dict(path_json):
    """Write the keywords to a json file
    
    Using this approach I don't have to upload data base schema information to github.
    
    """
    
    data_schema = {'histogram_column' : 'source column name',
                   'lable_suffix' : 'suffix',
                   'lable' : 'target column name',
                   'table' : 'field_data',
                   'factor' : 1,
                   'bin_start' : 0,
                   'bin_end' : 10,
                   'bin_count' : 10,
                   'where': 'lala > 0'}
    
    with open(path_json, 'w') as f:
        f.write(json.dumps(data_schema))
    
    return None
                

def create_sql_histogram(path_json):
    """Read the data base schema information from json and create sql_query"""
    
    with open(path_json, 'r') as f:
        data_schema = json.load(f)
        
    bins = np.linspace(data_schema['bin_start'], 
                       data_schema['bin_end'],
                       num=data_schema['bin_count'],
                       endpoint=True)
    
    sql_query = ''
    for idx, _ in enumerate(bins[0:-1]):
        if idx == (len(bins) - 2):
            union = 'ORDER BY idx;'
        else:
            union = 'UNION'
            
        sql_query = """%sSELECT
    '%.2f to %.2f %s' AS Bucket,
    %d AS idx,
    COUNT(%s) * %.15f AS "%s"
FROM %s
WHERE %s BETWEEN %f AND %f
%s
%s
""" % (sql_query,
       bins[idx],
       bins[idx+1],
       data_schema['lable_suffix'],
       idx,
       data_schema['histogram_column'],
       data_schema['factor'],
       data_schema['lable'],
       data_schema['table'],
       data_schema['histogram_column'],
       bins[idx],
       bins[idx+1],
       data_schema['where'],
       union) 
        
        
    return sql_query

In [12]:
my_query = create_sql_histogram('static_error.json')

print(my_query)
with open('static_error.sql', 'w') as f:
    f.write(my_query)

SELECT
    '0.00 to 0.20 mm' AS Bucket,
    0 AS idx,
    COUNT(staticerror) * 1.000000000000000 AS "static error [mm]"
FROM field_data
WHERE staticerror BETWEEN 0.000000 AND 0.200000
AND (epsabsminvehspd >= 10) AND (epssdsmodeact > 1) AND (epssdsmodeact < 3)
UNION
SELECT
    '0.20 to 0.40 mm' AS Bucket,
    1 AS idx,
    COUNT(staticerror) * 1.000000000000000 AS "static error [mm]"
FROM field_data
WHERE staticerror BETWEEN 0.200000 AND 0.400000
AND (epsabsminvehspd >= 10) AND (epssdsmodeact > 1) AND (epssdsmodeact < 3)
UNION
SELECT
    '0.40 to 0.60 mm' AS Bucket,
    2 AS idx,
    COUNT(staticerror) * 1.000000000000000 AS "static error [mm]"
FROM field_data
WHERE staticerror BETWEEN 0.400000 AND 0.600000
AND (epsabsminvehspd >= 10) AND (epssdsmodeact > 1) AND (epssdsmodeact < 3)
UNION
SELECT
    '0.60 to 0.80 mm' AS Bucket,
    3 AS idx,
    COUNT(staticerror) * 1.000000000000000 AS "static error [mm]"
FROM field_data
WHERE staticerror BETWEEN 0.600000 AND 0.800000
AND (epsabsminvehs